Set up there are a bunch of modules that neewd to be installed prior to running this notebook 
Download and install Java OpenJDK 8 https://www.azul.com/downloads/zulu-community/?version=java-8-lts&package=jdk-fx

Download and install Maven https://maven.apache.org/
add the maven bin folder to the system path variable

set up a new environment with mamba
conda install mamba -n base -c conda-forge
mamba create -n pyimagej -c conda-forge pyimagej openjdk=8
conda activate pyimagej


import all the neccesary modules

In [2]:
import os

Set up paths so apps and files

In [5]:
#apps
thermal_SDK=r'F:\IanDennis\Downloads\DJI\dji_thermal_sdk_v1.4_20220929\utility\bin\windows\release_x64\dji_irp'
exif_tools=r'F:\IanDennis\Downloads\EXIF_Tools\exiftool'

#images
image_Folder=r'F:\IanDennis\Other\Thermal\Images\Rjpeg'
image_out_Folder=r'F:\IanDennis\Other\Thermal\Images\Radiometric'

convert all Rjpeg to radiometic Raw

In [8]:

img_list = os.listdir(image_Folder)

for img in img_list:
    
    cmd = f"{thermal_SDK} -s {image_Folder}\{img} -a measure -o {image_out_Folder}\{img.split('.')[0]}.raw --measurefmt float32"
    os.system(cmd)
    

convert all Raw files to Tif

This is the batch that I ran within imageJ - could not yet automate within python

dir1 = "F:\\IanDennis\\Other\\Thermal\\Images\\Radiometric\\"; 
dir2 = "F:\\IanDennis\\Other\\Thermal\\Images\\Radiometric\\Out\\";
list = getFileList(dir1);
setBatchMode(true); 
for (i=0; i<list.length-1; i++) {
    run("Raw...", "open="+dir1+list[i]+" image=[32-bit Real] width=640 height=512 offset=0 number=1 gap=0 little-endian");
    saveAs("Tiff", dir2+list[i]);
    close();
}

In [ ]:
import imagej
# ij = imagej.init("2.5.0")
ij = imagej.init()
print(f"ImageJ version: {ij.getVersion()}")

macro = """
#@ String inDir
#@ String outDir

#@output Object greeting
list = getFileList(inDir);
setBatchMode(true); 

greeting="Start";

for (i=0; i<list.length; i++) {
    image_url = inDir+list[i];
    out_url = outDir+list[i];
    run("Raw...", "open="+image_url+" image=[32-bit Real] width=640 height=512 offset=0 number=1 gap=0 little-endian");
    saveAs("Tiff", out_url);
    close();



}

"""
args = {
    'inDir': 'F:\\IanDennis\\Other\\Thermal\\Images\\Radiometric\\Test_raw\\',
    'outDir': 'F:\\IanDennis\\Other\\Thermal\\Images\\Radiometric\\Test_Out\\'
}
result = ij.py.run_macro(macro, args)
print(result.getOutput('greeting'))

copy tags from original Jpegs to Tiffs

In [11]:
for img in img_list:
    
    cmd = f"{exif_tools} -tagsfromfile {image_Folder}\{img} -all:all {image_out_Folder}\Out\{img.split('.')[0]}.tif"
    # print(cmd)
    os.system(cmd)